# MTA Analysis: Data Analysis & Visualization

#### 1. Figure out what plotting library we are going to use.

#### 2. Time of Day Trends

#### 3. Day of Week Trends

#### 4. Days to Avoid

#### 5. Long Term Time Trends

#### 6. Map of Highest Impact Areas

recommend specific booths to stand near since different booths service different isolated access points in the station